In [1]:
import pandas as pd
import ast
from collections import Counter
import os
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate, KFold

import spacy
from sklearn.ensemble import RandomForestRegressor




In [2]:
os.listdir('../data/')

['listings.csv.gz',
 'reviews.csv',
 '.DS_Store',
 'listings0130.xlsx',
 'listings.csv',
 'listings_copy.xlsx',
 'listings_with_amenities_score.xlsx',
 'reviews.csv (1).gz']

In [3]:
df = pd.read_excel('../data/listings_copy.xlsx')

In [4]:
df

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2818,https://www.airbnb.com/rooms/2818,20220106171342,2022-01-07,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi<br />...,"Indische Buurt (""Indies Neighborhood"") is a ne...",https://a0.muscache.com/pictures/10272854/8dcc...,3159,https://www.airbnb.com/users/show/3159,...,4.98,4.68,4.82,0363 5F3A 5684 6750 D14D,t,1,0,1,0,1.83
1,20168,https://www.airbnb.com/rooms/20168,20220106171342,2022-01-06,Studio with private bathroom in the centre 1,17th century Dutch townhouse in the heart of t...,Located just in between famous central canals....,https://a0.muscache.com/pictures/69979628/fd6a...,59484,https://www.airbnb.com/users/show/59484,...,4.62,4.87,4.49,0363 CBB3 2C10 0C2A 1E29,t,2,0,2,0,2.35
2,27886,https://www.airbnb.com/rooms/27886,20220106171342,2022-01-06,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,"Central, quiet, safe, clean and beautiful.",https://a0.muscache.com/pictures/02c2da9d-660e...,97647,https://www.airbnb.com/users/show/97647,...,4.93,4.90,4.80,0363 974D 4986 7411 88D8,t,1,0,1,0,1.87
3,28871,https://www.airbnb.com/rooms/28871,20220106171342,2022-01-06,Comfortable double room,<b>The space</b><br />In a monumental house ri...,"Flower market , Leidseplein , Rembrantsplein",https://a0.muscache.com/pictures/160889/362340...,124245,https://www.airbnb.com/users/show/124245,...,4.94,4.97,4.82,0363 607B EA74 0BD8 2F6F,f,2,0,2,0,2.69
4,29051,https://www.airbnb.com/rooms/29051,20220106171342,2022-01-06,Comfortable single room,This room can also accomodate 2 people. For a...,the street is quite lively especially on weeke...,https://a0.muscache.com/pictures/162009/bd6be2...,124245,https://www.airbnb.com/users/show/124245,...,4.91,4.87,4.75,0363 607B EA74 0BD8 2F6F,f,2,0,2,0,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5592,54115977,https://www.airbnb.com/rooms/54115977,20220106171342,2022-01-07,Spacious & Bright Appartement close to Vondelpark,Deze stijlvolle accommodatie is perfect voor g...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,6370784,https://www.airbnb.com/users/show/6370784,...,NaN,NaN,NaN,0363 5743 9133 3628 37CA,t,1,1,0,0,NaN
5593,54125298,https://www.airbnb.com/rooms/54125298,20220106171342,2022-01-06,Luxury apartment in city centre,This two-floor apartment has been fully renova...,"The apartment is perfectly located, in the pop...",https://a0.muscache.com/pictures/miso/Hosting-...,21081740,https://www.airbnb.com/users/show/21081740,...,NaN,NaN,NaN,0363 35A7 7EB7 3D9C 96A3,t,1,1,0,0,NaN
5594,54126733,https://www.airbnb.com/rooms/54126733,20220106171342,2022-01-06,Mediterraans design loft in de Pijp Amsterdam,Luxe design loft midden in de Pijp. Heel dicht...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,11632814,https://www.airbnb.com/users/show/11632814,...,NaN,NaN,NaN,0363 6FDC 0E33 44C1 941C,f,1,1,0,0,NaN
5595,54134640,https://www.airbnb.com/rooms/54134640,20220106171342,2022-01-06,Cosy & peaceful sanctuary in Amsterdam near Am...,You're warmly welcomed in my cosy sanctuary in...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,418516420,https://www.airbnb.com/users/show/418516420,...,NaN,NaN,NaN,0363 2DC5 9BFE 700E B647,f,1,1,0,0,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5597 entries, 0 to 5596
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            5597 non-null   int64         
 1   listing_url                                   5597 non-null   object        
 2   scrape_id                                     5597 non-null   int64         
 3   last_scraped                                  5597 non-null   datetime64[ns]
 4   name                                          5597 non-null   object        
 5   description                                   5589 non-null   object        
 6   neighborhood_overview                         3983 non-null   object        
 7   picture_url                                   5597 non-null   object        
 8   host_id                                       5597 non-null   int64 

#### Converting string representation of lists to lists

In [6]:
df['amenities'] = df.apply(lambda x: ast.literal_eval(x['amenities']), axis=1)

In [7]:
amenities_list = df['amenities'].to_list()
all_amenities = [item for item in (l for l in amenities_list)]
all_amenities = [item for sublist in amenities_list for item in sublist]
print(len(all_amenities))
print(len(set(all_amenities)))

151220
1016


In [8]:
count_amenities = Counter(all_amenities)
count_amenities.most_common()[:20]

[('Wifi', 5400),
 ('Essentials', 5324),
 ('Smoke alarm', 4936),
 ('Heating', 4849),
 ('Hangers', 4448),
 ('Hair dryer', 4415),
 ('Hot water', 4327),
 ('Kitchen', 4267),
 ('Long term stays allowed', 4223),
 ('Iron', 4090),
 ('Shampoo', 3973),
 ('Dishes and silverware', 3904),
 ('Dedicated workspace', 3889),
 ('Coffee maker', 3707),
 ('Refrigerator', 3676),
 ('Washer', 3489),
 ('Bed linens', 3326),
 ('Cooking basics', 3207),
 ('Carbon monoxide alarm', 3033),
 ('Fire extinguisher', 2991)]

In [9]:
unique_amenities = list(set(all_amenities))

In [10]:
print(len([u for u in unique_amenities if 'wifi' in u.lower()]))

91


In [11]:
print([u for u in unique_amenities if 'shampoo' in u.lower()])

['Shampoo, bodywash and hand wash is from Rituals Hammam body soap', 'Meraki shampoo', 'Savoir-Faire shampoo', 'Marie Stella Maris of Amsterdam shampoo', 'Eco shampoo shampoo', 'Rituals  shampoo', 'depends shampoo', 'eco shampoo', 'Spa Collection shampoo', 'Various shampoo', 'Eco-friendly shampoo', 'Andrelon  shampoo', 'various shampoo', 'eco weleda shampoo', 'Marcel  shampoo', 'Savoire Fair shampoo', 'Marie Stella Maris luxury brand shampoo', 'Standard shampoo', 'Marie Stella Maris shampoo', 'Dove or Baby shampoo shampoo', 'The Spa Collection shampoo', 'Bio shampoo', 'Stella Maris shampoo', 'Dove shampoo', 'Kiehls shampoo', 'Marie-Stella-Maris shampoo', 'Gfl Amenities shampoo', 'L’Oréal Elvive shampoo', 'Hema shampoo', 'rituals shampoo', 'Syoss shampoo', 'Organic shampoo shampoo', 'organic shampoo shampoo', 'Australien shampoo', 'Rituals shampoo', 'MARIE-STELLA-MARIS shampoo', 'Eco shampoo', 'Etos shampoo', 'andrelon shampoo', 'Weleda or similar  shampoo', 'Love Beauty and Planet ( ve

In [12]:
def get_common_substrings_count(l1, l2):
    
    """l1 contains a list of substrings, l2 contains strings. 
    This function returns the count of strings in l1 that are present in l2"""
    
    item_count = 0
    for item in l1:
        
        result = sum(item in l for l in l2)
        
        item_count+=result
    return item_count

In [13]:
def get_amenities_scores(df, amenities_col='amenities', 
                        safety_list=['wifi', 'essentials', 'alarm', 'Fire extinguisher'], 
                        comfort_list=['heating', 'hot water', 'linen', 'refrigerator', 'hair dryer', 'kitchen', 'cooking', 'hangers'], 
                        frill_list=['shampoo', 'soap', 'iron', 'coffee', 'workspace'], 
                        safety_points=10, 
                        comfort_points=7, 
                        frill_points=4):
    
    df['safety_score'] = 0
    df['comfort_score'] = 0
    df['frill_score'] = 0
    df['total_amenities_score'] = 0
        
    df['safety_score'] = df.apply(lambda x: get_common_substrings_count(safety_list, x[amenities_col]) * safety_points, axis=1)
    df['comfort_score'] = df.apply(lambda x: get_common_substrings_count(comfort_list, x[amenities_col]) * comfort_points, axis=1)
    df['frill_score'] = df.apply(lambda x: get_common_substrings_count(frill_list, x[amenities_col]) * frill_points, axis=1)
    
    df['total_amenities_score'] = df.apply(lambda x: x['safety_score'] + x['comfort_score'] + x['frill_score'], axis=1)
    
    return df    

#### Getting amenities scores of columns using the functions defined above

In [14]:
df = get_amenities_scores(df)

In [15]:
df

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,safety_score,comfort_score,frill_score,total_amenities_score
0,2818,https://www.airbnb.com/rooms/2818,20220106171342,2022-01-07,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi<br />...,"Indische Buurt (""Indies Neighborhood"") is a ne...",https://a0.muscache.com/pictures/10272854/8dcc...,3159,https://www.airbnb.com/users/show/3159,...,t,1,0,1,0,1.83,40,7,12,59
1,20168,https://www.airbnb.com/rooms/20168,20220106171342,2022-01-06,Studio with private bathroom in the centre 1,17th century Dutch townhouse in the heart of t...,Located just in between famous central canals....,https://a0.muscache.com/pictures/69979628/fd6a...,59484,https://www.airbnb.com/users/show/59484,...,t,2,0,2,0,2.35,30,7,4,41
2,27886,https://www.airbnb.com/rooms/27886,20220106171342,2022-01-06,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,"Central, quiet, safe, clean and beautiful.",https://a0.muscache.com/pictures/02c2da9d-660e...,97647,https://www.airbnb.com/users/show/97647,...,t,1,0,1,0,1.87,30,14,8,52
3,28871,https://www.airbnb.com/rooms/28871,20220106171342,2022-01-06,Comfortable double room,<b>The space</b><br />In a monumental house ri...,"Flower market , Leidseplein , Rembrantsplein",https://a0.muscache.com/pictures/160889/362340...,124245,https://www.airbnb.com/users/show/124245,...,f,2,0,2,0,2.69,30,7,0,37
4,29051,https://www.airbnb.com/rooms/29051,20220106171342,2022-01-06,Comfortable single room,This room can also accomodate 2 people. For a...,the street is quite lively especially on weeke...,https://a0.muscache.com/pictures/162009/bd6be2...,124245,https://www.airbnb.com/users/show/124245,...,f,2,0,2,0,3.99,30,7,0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5592,54115977,https://www.airbnb.com/rooms/54115977,20220106171342,2022-01-07,Spacious & Bright Appartement close to Vondelpark,Deze stijlvolle accommodatie is perfect voor g...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,6370784,https://www.airbnb.com/users/show/6370784,...,t,1,1,0,0,NaN,20,14,4,38
5593,54125298,https://www.airbnb.com/rooms/54125298,20220106171342,2022-01-06,Luxury apartment in city centre,This two-floor apartment has been fully renova...,"The apartment is perfectly located, in the pop...",https://a0.muscache.com/pictures/miso/Hosting-...,21081740,https://www.airbnb.com/users/show/21081740,...,t,1,1,0,0,NaN,10,14,12,36
5594,54126733,https://www.airbnb.com/rooms/54126733,20220106171342,2022-01-06,Mediterraans design loft in de Pijp Amsterdam,Luxe design loft midden in de Pijp. Heel dicht...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,11632814,https://www.airbnb.com/users/show/11632814,...,f,1,1,0,0,NaN,20,0,0,20
5595,54134640,https://www.airbnb.com/rooms/54134640,20220106171342,2022-01-06,Cosy & peaceful sanctuary in Amsterdam near Am...,You're warmly welcomed in my cosy sanctuary in...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,418516420,https://www.airbnb.com/users/show/418516420,...,f,1,1,0,0,NaN,30,14,4,48


In [16]:
df = df.dropna(subset=['reviews_per_month'])

In [17]:
df = df.dropna(subset=['price'])

In [18]:
df = df.dropna(subset=['total_amenities_score'])



In [19]:
df = df.dropna(subset=['host_acceptance_rate'])
df = df.dropna(subset=['host_response_rate'])
df = df.dropna(subset=['bedrooms'])
df = df.dropna(subset=['beds'])
df = df.dropna(subset=['description'])
df = df.dropna(subset=['review_scores_rating'])





In [20]:
list(df.columns)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'description',
 'neighborhood_overview',
 'picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'latitude',
 'longitude',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bathrooms_text',
 'bedrooms',
 'beds',
 'amenities',
 'price',
 'minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availa

In [21]:
len(df)

3237

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3237 entries, 0 to 5571
Data columns (total 78 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            3237 non-null   int64         
 1   listing_url                                   3237 non-null   object        
 2   scrape_id                                     3237 non-null   int64         
 3   last_scraped                                  3237 non-null   datetime64[ns]
 4   name                                          3237 non-null   object        
 5   description                                   3237 non-null   object        
 6   neighborhood_overview                         2377 non-null   object        
 7   picture_url                                   3237 non-null   object        
 8   host_id                                       3237 non-null   int64 

In [23]:
df["instant_bookable"]

0       t
1       t
2       t
3       f
4       f
       ..
5554    t
5558    t
5559    f
5567    f
5571    t
Name: instant_bookable, Length: 3237, dtype: object

In [24]:
from sklearn.metrics import SCORERS

In [25]:
list(SCORERS.keys())

['explained_variance',
 'r2',
 'max_error',
 'neg_median_absolute_error',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_root_mean_squared_error',
 'neg_mean_poisson_deviance',
 'neg_mean_gamma_deviance',
 'accuracy',
 'top_k_accuracy',
 'roc_auc',
 'roc_auc_ovr',
 'roc_auc_ovo',
 'roc_auc_ovr_weighted',
 'roc_auc_ovo_weighted',
 'balanced_accuracy',
 'average_precision',
 'neg_log_loss',
 'neg_brier_score',
 'adjusted_rand_score',
 'rand_score',
 'homogeneity_score',
 'completeness_score',
 'v_measure_score',
 'mutual_info_score',
 'adjusted_mutual_info_score',
 'normalized_mutual_info_score',
 'fowlkes_mallows_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'jaccard',
 'jaccard_macro',
 'jaccard_m

In [26]:
#!python -m spacy download en_core_web_lg

#### Load the spacy langauge model for english

In [27]:
nlp = spacy.load("en_core_web_lg")

#### Using spacy to clean the description text by lemmatizing, keeping only alphabets, removing stop words and making the text lower case

In [28]:
%%time
df['clean_description']=df["description"].apply(lambda x: " ".join(token.lemma_ for token in nlp(x) if token.lemma_.lower() not in nlp.Defaults.stop_words and token.is_alpha))

CPU times: user 1min 5s, sys: 5.5 s, total: 1min 11s
Wall time: 1min 11s


#### Getting a vector representation of each description text using spacy's Word2Vec model. The spacy model uses a 300 dimensional vector representation for every word and if nlp.vector is applied to a sentence or span of text, the average vector of all the words in the span is calculated

In [29]:
%%time
df['description_vector'] = [nlp(doc).vector.reshape(1,-1) for doc in df['clean_description'].tolist()]

CPU times: user 33.6 s, sys: 1.43 s, total: 35 s
Wall time: 35.2 s


In [30]:
np.shape(np.concatenate(df['description_vector'].to_list()))

(3237, 300)

In [31]:
df['property'] = [p[-1].lower() for p in df['property_type'].str.split(' in |Entire ')]

In [32]:
df.groupby('property')['price'].agg(['mean', 'median', 'count', 'min', 'max', 'std']).sort_values('median')

,mean,median,count,min,max,std
property,,,,,,
hostel,53.000000,36.0,27,18,147,34.023747
camper/rv,74.000000,74.0,1,74,74,NaN
barn,85.000000,85.0,1,85,85,NaN
farm stay,88.428571,87.0,7,82,103,7.161404
boutique hotel,117.825688,91.0,109,31,980,122.885361
nature lodge,93.000000,93.0,1,93,93,NaN
bungalow,95.000000,95.0,1,95,95,NaN
guest suite,115.555556,95.0,81,44,371,64.390023
private room,104.666667,96.5,6,55,180,44.572039


In [33]:
df['property'] = df['property'].astype('category')

#### Creating columns for each property type

In [34]:
df = pd.get_dummies(df, columns=['property'])
df.drop(columns=['property_type'], inplace=True)

#### Collecting binary property columns

In [35]:
property_cols = [p for p in df.columns if p.startswith('property')]

In [36]:
X = df[["reviews_per_month", 
              "safety_score", "comfort_score", "frill_score",  "host_response_rate", 
              "host_acceptance_rate", "bedrooms", "beds", "review_scores_rating", 
        "total_amenities_score", "safety_score", "comfort_score", "frill_score"]+property_cols]

y = df["price"]

#### Combining other features with the word vector features

In [37]:
X_combined = np.concatenate((np.concatenate(df['description_vector'].to_list()), X.values), 1)

In [38]:
%%time
from sklearn.model_selection import cross_validate, KFold

kfold = KFold(n_splits=10, shuffle=True, random_state=5)
scores = cross_validate(Lasso(), X_combined, y, cv=kfold,
                         scoring=('r2'), return_estimator=True
                         )
metrics_df_l = pd.DataFrame(scores)
metrics_df_l.describe()

CPU times: user 2.23 s, sys: 944 ms, total: 3.17 s
Wall time: 290 ms


,fit_time,score_time,test_score
count,10.000000,10.000000,10.000000
mean,0.024129,0.000781,0.363336
std,0.005924,0.000188,0.091311
min,0.018598,0.000682,0.129337
25%,0.020972,0.000706,0.352738
50%,0.021530,0.000710,0.386878
75%,0.024168,0.000754,0.416707
max,0.036920,0.001309,0.443078


In [39]:
%%time

kfold = KFold(n_splits=10, shuffle=True, random_state=5)
scores = cross_validate(Ridge(), X_combined, y, cv=kfold,
                         scoring=('r2'), return_estimator=True
                         )
metrics_df_r = pd.DataFrame(scores)
metrics_df_r.describe()

CPU times: user 1.57 s, sys: 536 ms, total: 2.11 s
Wall time: 152 ms


,fit_time,score_time,test_score
count,10.000000,10.000000,10.000000
mean,0.012187,0.000741,0.417534
std,0.002822,0.000059,0.091113
min,0.010493,0.000633,0.175537
25%,0.010597,0.000698,0.430333
50%,0.010955,0.000760,0.439817
75%,0.011656,0.000776,0.464610
max,0.019207,0.000819,0.490409


In [40]:
%%time

kfold = KFold(n_splits=10, shuffle=True, random_state=5)
scores = cross_validate(RandomForestRegressor(), X_combined, y, cv=kfold,
                         scoring=('r2'), return_estimator=True
                         )
metrics_df_rf = pd.DataFrame(scores)
metrics_df_rf.describe()

CPU times: user 6min 56s, sys: 1.24 s, total: 6min 57s
Wall time: 6min 57s


,fit_time,score_time,test_score
count,10.000000,10.000000,10.000000
mean,41.715671,0.013552,0.402775
std,1.402023,0.001082,0.072596
min,40.002459,0.012314,0.250319
25%,40.493885,0.012913,0.388810
50%,41.270521,0.013216,0.392764
75%,43.134908,0.014063,0.456106
max,43.517732,0.015889,0.503959


In [41]:
metrics_df_rf

,fit_time,score_time,estimator,test_score
0,43.517732,0.014312,"(DecisionTreeRegressor(max_features='auto', ra...",0.339636
1,43.138080,0.013288,"(DecisionTreeRegressor(max_features='auto', ra...",0.388550
2,43.125392,0.013144,"(DecisionTreeRegressor(max_features='auto', ra...",0.389593
3,43.327678,0.014654,"(DecisionTreeRegressor(max_features='auto', ra...",0.436496
4,41.303472,0.012875,"(DecisionTreeRegressor(max_features='auto', ra...",0.462643
5,40.726996,0.013316,"(DecisionTreeRegressor(max_features='auto', ra...",0.471025
6,41.237570,0.012697,"(DecisionTreeRegressor(max_features='auto', ra...",0.390680
7,40.002459,0.015889,"(DecisionTreeRegressor(max_features='auto', ra...",0.394848
8,40.416181,0.013026,"(DecisionTreeRegressor(max_features='auto', ra...",0.250319
9,40.361151,0.012314,"(DecisionTreeRegressor(max_features='auto', ra...",0.503959


In [42]:
scores["estimator"]

[RandomForestRegressor(),
 RandomForestRegressor(),
 RandomForestRegressor(),
 RandomForestRegressor(),
 RandomForestRegressor(),
 RandomForestRegressor(),
 RandomForestRegressor(),
 RandomForestRegressor(),
 RandomForestRegressor(),
 RandomForestRegressor()]

In [43]:
for idx,estimator in enumerate(scores['estimator']):
    print("Features sorted by their score for estimator {}:".format(idx))
    feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       #index = X_combined,
                                        columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importances[:10])


Features sorted by their score for estimator 0:
     importance
306    0.304615
300    0.040188
229    0.021742
307    0.021065
221    0.016219
23     0.012090
308    0.007683
191    0.007643
25     0.006640
202    0.006159
Features sorted by their score for estimator 1:
     importance
306    0.322046
300    0.036757
229    0.023320
307    0.017065
221    0.012795
308    0.011897
23     0.010294
290    0.007324
191    0.006223
10     0.005956
Features sorted by their score for estimator 2:
     importance
306    0.313553
300    0.038264
221    0.021218
307    0.017835
229    0.017423
23     0.013712
308    0.008736
25     0.007195
153    0.006205
150    0.005871
Features sorted by their score for estimator 3:
     importance
306    0.315862
300    0.036925
221    0.023484
307    0.022609
229    0.021355
23     0.009878
72     0.009059
308    0.006948
202    0.006590
11     0.005956
Features sorted by their score for estimator 4:
     importance
306    0.315143
300    0.036896
221    0